# Using OpenAI API to Peform Recommendations Based on Sentiment Analysis
We are already at the age of AI. Bringing the sentiment score only is not enough. Yes, we have the insights, but what good is insights if we don't have any good actions to follow through? With the availability of LLM, we can use it's power to generate recommendations for us based on the sentiment analysis that we have done. With this, we can provide prescriptive analytics and bring more value to our analysis. 

This notebook serves as a prototype to test whether we can use OpenAI API to generate recommendations for each sentiment analysis that we have done on petrol stations. The objective is:

1. The test the feasability of the concept - using OpenAI API to generate recommendations for each sentiment analysis
2. To track the usage of tokens during recommendation generation and ultimately the cost
3. Serves as testing platform before moving on to web applications - streamlit web app

In [21]:
#first, let's looad the data first
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

df = pd.read_csv('sentiment_results.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9898 entries, 0 to 9897
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       9898 non-null   int64  
 1   station_name     9898 non-null   object 
 2   place_name       9898 non-null   object 
 3   location         9898 non-null   object 
 4   author_name      9898 non-null   object 
 5   rating           9898 non-null   int64  
 6   text             9631 non-null   object 
 7   time             9898 non-null   int64  
 8   Language         9898 non-null   object 
 9   Sentiment        9898 non-null   object 
 10  Sentiment_Score  9898 non-null   float64
dtypes: float64(1), int64(3), object(7)
memory usage: 850.7+ KB


In [22]:
#let's look back at our data
df.head(10)

,Unnamed: 0,station_name,place_name,location,author_name,rating,text,time,Language,Sentiment,Sentiment_Score
0,0,TAMAN ANDA,Petronas Taman Anda,"4.629239,101.114431",Stephen ong,3,Another station charging higher than normal di...,1676626422,en,very negative,0.647881
1,1,TAMAN ANDA,Petronas Taman Anda,"4.629239,101.114431",Che Syaiful,4,Got petrol here,1726010026,en,very positive,0.429729
2,2,TAMAN ANDA,Petronas Taman Anda,"4.629239,101.114431",Mohammad Fithri Mohammad Sharifuddin,4,"most of the staff are very helpful, kind and a...",1601110289,en,very positive,0.549436
3,3,TAMAN ANDA,Petronas Taman Anda,"4.629239,101.114431",Stephen lee,5,Staff always with smiling face,1600260670,en,very positive,0.298948
4,4,TAMAN ANDA,Petronas Taman Anda,"4.629239,101.114431",Iqbal Hakim (Kim),5,"Its okay, typical petrol station.",1614015946,en,neutral,0.778628
5,5,PARIT BUNTAR,Caltex,"5.115475,100.484894",Mohd Ammar,5,One of two choices you can opt when in parit b...,1542117724,en,positive,0.487942
6,6,PARIT BUNTAR,Caltex,"5.115475,100.484894",Amirul Husnisham,5,"One of the best Petrol Station in Malaysia, th...",1592062531,en,very positive,0.890043
7,7,PARIT BUNTAR,Caltex,"5.115475,100.484894",moon lucas,5,Most clean petrol station. Of course alot snac...,1569220138,en,positive,0.412165
8,8,PARIT BUNTAR,Caltex,"5.115475,100.484894",onn keong leong,5,Very friendly owner. Treat everyone like a clo...,1496890312,en,very positive,0.636795
9,9,PARIT BUNTAR,Caltex,"5.115475,100.484894",eimoc mirror,4,Good and clean petrol stations. Strategic loca...,1489928693,en,very positive,0.489335


From the data above, we can see that each stations have multiple ratings, reviews and sentiment score. It is better that we combine and aggregate all of this before we include it in our prompt. 

In [23]:
#first, let's preprocess it. our text column is in object, it should be in string, before we can join it
df['text']= df['text'].fillna('')

df['text'] = df['text'].astype(str)
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9898 entries, 0 to 9897
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Unnamed: 0       9898 non-null   int64  
 1   station_name     9898 non-null   object 
 2   place_name       9898 non-null   object 
 3   location         9898 non-null   object 
 4   author_name      9898 non-null   object 
 5   rating           9898 non-null   int64  
 6   text             9898 non-null   object 
 7   time             9898 non-null   int64  
 8   Language         9898 non-null   object 
 9   Sentiment        9898 non-null   object 
 10  Sentiment_Score  9898 non-null   float64
dtypes: float64(1), int64(3), object(7)
memory usage: 850.7+ KB


In [24]:
# Group the data by 'Station Name', and aggregate
df_agg = df.groupby('station_name').agg({
    'rating': 'mean',  # Average rating
    'Sentiment_Score': 'mean',  # Average sentiment score
    'text': lambda x: ' '.join(x)  # Combine all reviews into a single string
}).reset_index()

df_agg

,station_name,rating,Sentiment_Score,text
0,AIR HITAM,4.200000,0.453347,Excellent Gas Station with Clean Amenities and...
1,AIR KEROH 3,4.600000,0.567572,A wonderful Petronas outlet after the Ayer Ker...
2,AJIL,3.200000,0.587329,Standard petrol pump in a suburban area in the...
3,AL MUKTAFI BILLAH SHAH,5.000000,0.638511,Friendly and very good service. Friendly and v...
4,ALAM DAMAI,3.400000,0.590713,Prefer there's 1 old staff works there longer ...
...,...,...,...,...
1083,UTP,3.800000,0.552209,Car always give low fuel economy..may be amoun...
1084,WAKAF BHARU,4.000000,0.569747,Tempat okay. Except that staff is not proactiv...
1085,WANGSA MELAWATI,4.466667,0.445799,"a convenient place to buy snacks, drinks, and ..."
1086,YAN,3.800000,0.460212,"Internet connection is quite slow hence, take ..."


Alright, since we have succefully aggregate our dataset to diplay only the consolidated score and review, let's use that info and pass it to OpenAI. We will use gpt-3.5-turbo as our model to minimize cost. We will pass specific prompt to ensure that it can provide recommendations based on rating, sentiment score and the reviews

In [27]:
import openai
import os
from dotenv import load_dotenv

# Load environment variables from a .env file (optional, if you use one)
load_dotenv()

# Set OpenAI API key using environment variable or hardcoded value
openai.api_key = os.getenv("OPENAI_API_KEY")

# Function to generate business recommendations using the OpenAI API
def generate_business_recommendation(station_name, avg_rating, avg_sentiment_score, combined_reviews):
    # Prepare the prompt
    prompt = f"""
As a business analyst, analyze the following data about the fuel station '{station_name}':
- Average Rating: {avg_rating}/5
- Average Sentiment Score: {avg_sentiment_score}
- Combined Customer Reviews: '{combined_reviews}'
Based on this information, provide actionable recommendations for the business to improve this station's performance and customer satisfaction.
    """
    
    # Make the API request using gpt-3.5-turbo
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an expert business analyst who provides actionable recommendations to improve business performance based on customer feedback."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.7
    )
    
    # Extract the recommendation text from the response
    recommendation = response['choices'][0]['message']['content'].strip()
    return recommendation


In [28]:
# Apply the API to generate business recommendations for the first 10 stations
df_agg.loc[:9, 'Business_Recommendation'] = df_agg.iloc[:10].apply(
    lambda row: generate_business_recommendation(
        row['station_name'],
        round(row['rating'], 2),  # Round the average rating to 2 decimal places
        round(row['Sentiment_Score'], 2),  # Round the average sentiment score to 2 decimal places
        row['text']
    ),
    axis=1
)

# Display the results for the first 10 rows
print(df_agg.loc[:9, ['station_name', 'rating', 'Sentiment_Score', 'text', 'Business_Recommendation']])

# Optional: Save the first 10 results to a CSV file
df_agg.loc[:9].to_csv('first_10_station_business_recommendations.csv', index=False)


             station_name  rating  Sentiment_Score  \
0               AIR HITAM     4.2         0.453347   
1             AIR KEROH 3     4.6         0.567572   
2                    AJIL     3.2         0.587329   
3  AL MUKTAFI BILLAH SHAH     5.0         0.638511   
4              ALAM DAMAI     3.4         0.590713   
5             ALAM SUTERA     2.8         0.614468   
6                    ALMA     2.4         0.625769   
7              ALOR MERAH     3.7         0.506146   
8              AMPANGAN 2     4.0         0.525470   
9          AMPANGAN UTAMA     3.4         0.557800   

                                                text  \
0  Excellent Gas Station with Clean Amenities and...   
1  A wonderful Petronas outlet after the Ayer Ker...   
2  Standard petrol pump in a suburban area in the...   
3  Friendly and very good service. Friendly and v...   
4  Prefer there's 1 old staff works there longer ...   
5  PARKING = Parking is limited,\nSERVICE = The s...   
6  Rude, irre

In [29]:
df_agg['Business Recommendation'].unique()

array(["Based on the data and customer feedback provided for the fuel station 'AIR HITAM', here are some actionable recommendations to improve performance and customer satisfaction:\n\n1. **Maintain the Cleanliness Standards**: Continue prioritizing the cleanliness and maintenance of the amenities, especially the restrooms. Consistent cleanliness contributes significantly to customer satisfaction and loyalty.\n\n2. **Enhance Food and Beverage Selection**: While the feedback mentions a wide variety of food and beverage items, consider regularly updating and diversifying the selection to cater to evolving customer preferences. Conduct periodic surveys or feedback collection to understand customer preferences better.\n\n3. **Optimize Parking and Maneuverability**: Since the station is spacious with ample parking, ensure efficient traffic flow and parking organization to enhance customer convenience. Consider",
       "Based on the data and customer feedback for the fuel station 'AIR KEROH

In [ ]:
import streamlit as st
import pandas as pd
import openai
import pydeck as pdk

# Set up OpenAI API key
openai.api_key = 'your_openai_api_key_here'

# Sample data for stations where location is a string "(latitude,longitude)"
data = {
    'Station Name': ['Station A', 'Station B', 'Station C'],
    'Location': ['(3.139003,101.686855)', '(3.157850,101.712616)', '(3.130700,101.650522)'],
    'Rating': [4.2, 3.8, 4.5],
    'Sentiment Score': [0.9, 0.7, 0.85],
    'Review Text': [
        "Great service, but the fuel prices are a bit high.",
        "Decent location but customer service needs improvement.",
        "Clean and well-maintained, with friendly staff."
    ]
}

df = pd.DataFrame(data)

# Split 'Location' column into 'Latitude' and 'Longitude'
df[['Latitude', 'Longitude']] = df['Location'].str.extract(r'\((.*),(.*)\)')
df['Latitude'] = pd.to_numeric(df['Latitude'])
df['Longitude'] = pd.to_numeric(df['Longitude'])

# Streamlit layout
st.title("Fuel Station Dashboard")

# Show the stations on a map
st.subheader("Map of Stations")
st.map(df[['Latitude', 'Longitude']])

# Dropdown to select a station
station = st.selectbox("Select a Station", df['Station Name'])

# Display sentiment score and rating for the selected station
selected_station_data = df[df['Station Name'] == station].iloc[0]
st.write(f"**Rating**: {selected_station_data['Rating']}")
st.write(f"**Sentiment Score**: {selected_station_data['Sentiment Score']}")

# Button to call GPT-3.5 for a recommendation
if st.button('Get GPT-3.5 Business Recommendation'):
    
    def generate_business_recommendation(station_name, avg_rating, avg_sentiment_score, combined_reviews):
        # Prepare the prompt
        prompt = f"""
As a business analyst, analyze the following data about the fuel station '{station_name}':
- Average Rating: {avg_rating}/5
- Average Sentiment Score: {avg_sentiment_score}
- Combined Customer Reviews: '{combined_reviews}'
Based on this information, provide actionable recommendations for the business to improve this station's performance and customer satisfaction.
        """
        
        # Call GPT-3.5 API
        response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "You are an expert business analyst who provides actionable recommendations to improve business performance based on customer feedback."},
                {"role": "user", "content": prompt}
            ],
            max_tokens=300,  # Increased to 300 tokens
            temperature=0.7
        )
        
        # Extract the recommendation
        recommendation = response['choices'][0]['message']['content'].strip()
        return recommendation
    
    # Call the function to generate the recommendation
    recommendation = generate_business_recommendation(
        selected_station_data['Station Name'],
        selected_station_data['Rating'],
        selected_station_data['Sentiment Score'],
        selected_station_data['Review Text']
    )
    
    # Display the recommendation
    st.subheader(f"GPT-3.5 Recommendation for {selected_station_data['Station Name']}")
    st.write(recommendation)
